In [1]:
from src.requirements import *

## Audio Preprocessing
- resample
- stereo to mono
- normalize
- plot

In [2]:
class AudioDataset(Dataset):
    def __init__(self, audio_path, target_sr=16000, transform=None):
        self.audio_path = audio_path
        self.target_sr = target_sr
        self.transform = transform
        self.file_list = []
        self._gather_files()

    def _gather_files(self):
        for dir1 in os.listdir(self.audio_path):
            if dir1.endswith('.tsv'):
                continue
            subdir = os.path.join(self.audio_path, dir1)
            for dir2 in tqdm(os.listdir(subdir)):
                if dir2.endswith('.tsv'):
                    continue
                path = os.path.join(subdir, dir2)
                self.file_list.append(path)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        waveform, sr = sf.read(path, always_2d=True)
        waveform = torch.Tensor(waveform.T)

        if sr != self.target_sr:
            resampler = T.Resampler(orig_freq=sr, new_freq=self.target_sr)
            waveform = resampler(waveform)

        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        waveform = waveform / (waveform.abs().max() + 1e-8)

        if self.transform:
            waveform = self.transform(waveform)

        return waveform.squeeze(0)

In [3]:
audio_path = 'data'
data = AudioDataset(audio_path, target_sr=16000)

100%|█████████████████████████████████████████████████████████████████████████████| 618/618 [00:00<00:00, 56078.92it/s]


In [4]:
def collate_padding(batch):
    batch = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=0)
    batch = batch.unsqueeze(1)
    return batch

In [5]:
train_data = DataLoader(dataset=data, batch_size=16, collate_fn=collate_padding)

In [6]:
for batch in train_data:
    print(batch.shape)
    print(type(batch))
    for item in batch:
        print(item.shape)
        break
    break

torch.Size([16, 1, 99200])
<class 'torch.Tensor'>
torch.Size([1, 99200])
